In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/final_R1/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 2e-4
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(4)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=ACERNetwork(sess,env,"global",network_type=network_t)
        agent = ACERNetwork(sess,env,str(0),G_Agent, network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.24605832 0.2475322  0.25556323 0.25084633]]
[[0.25740805 0.24800438 0.25360066 0.24098687]]
[[0.25740805 0.24800438 0.25360066 0.24098687]]
[[0.25740805 0.24800438 0.25360066 0.24098687]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = False, save_path=TB_DIR, replay_ratio=0):
        
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)
            

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", 20, 0.99, 1000000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), 20, 0.99, 1000000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()
                
            # Join the processes, so we get this thread back.
            coord.join(processes)


In [ ]:
tf.reset_default_graph()
start_method(replay_ratio=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1  using  1  per online step
worker_2worker_3  using  1  per online step
 worker_4  using  1  per online step
worker_5 using   using  1  per online step
worker_6  using  1  per online step
worker_7  using  1  per online step
worker_8  using  1  per online step
 1worker_9 worker_10  using  1  per online step
worker_11  using  1  per online step
 per online step worker_12  using  1  per online step
 using  1  per online step
worker_13  using  1  per online step
worker_14  using  1  per online step

worker_15  using  1  per online step


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  385  Running/Max:  1.0 1.0  Frames: 13
pi: [[0.25876406 0.2451112  0.2514391  0.24468565]]
Saving Model
Worker  worker_eval At  10202  Running/Max:  1.4226778050919033 4.0  Frames: 321
pi: [[0.00286972 0.33325392 0.6478814  0.01599497]]
Saving Model
Worker  worker_eval At  20061  Running/Max:  1.7259240577986699 5.0  Frames: 628
pi: [[0.16001788 0.34401578 0.36591637 0.13004996]]
Saving Model
Worker  worker_eval At  30186  Running/Max:  1.9774386652889937 5.0  Frames: 942
pi: [[0.2501435  0.29518393 0.26506087 0.18961166]]
Saving Model
Worker  worker_eval At  40317  Running/Max:  2.34031831362687 5.0  Frames: 1257
pi: [[0.25297216 0.3156498  0.27565676 0.15572134]]
Saving Model
Worker  worker_eval At  50159  Running/Max:  2.033853557807812 5.0  Frames: 1562
pi: [[0.19525984 0.35392678 0.21716818 0.23364525]]
Saving Model
Worker  worker_eval At  60424  Running/Max:  2.6030235378584643 5.0  Frames: 1879
pi: [[0.17863987 0.39842847 0.18127786 0.24165381]]
Saving Mo

In [6]:
TB_DIR = "saves/breakout/final_R3/"
tf.reset_default_graph()
start_method(replay_ratio=3)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1  using  1  per online step
worker_2  using  1  per online step
worker_3  using  1  per online step
worker_4  using  1  per online step
worker_5  using  1  per online step
worker_6  using  1  per online step
worker_7  using  1  per online step
worker_8  using  1  per online step
worker_9  using  1  per online step
worker_10  using  1  per online step
worker_11  using  1  per online step
worker_12worker_13  using  1  per online step
  using worker_14  using  1  per online step
worker_15  using  1  per online step
 1  per online step


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  237  Running/Max:  1.1 2.0  Frames: 237
pi: [[0.25427207 0.25346285 0.24547124 0.24679382]]
Saving Model
Worker  worker_eval At  10253  Running/Max:  1.158968134867575 4.0  Frames: 10253
pi: [[0.1957629  0.0860133  0.25290048 0.46532333]]
Saving Model
Worker  worker_eval At  20271  Running/Max:  1.6398080927452432 6.0  Frames: 18470
pi: [[0.25152594 0.2973084  0.2538016  0.19736409]]
Saving Model
Worker  worker_eval At  30337  Running/Max:  2.0335907861172635 6.0  Frames: 23502
pi: [[0.2032693  0.26567182 0.3940758  0.13698314]]
Saving Model
Worker  worker_eval At  40213  Running/Max:  1.896020507946321 6.0  Frames: 28442
pi: [[0.33052227 0.3448918  0.22588773 0.09869818]]
Saving Model
Worker  worker_eval At  50891  Running/Max:  3.4839256298414285 8.0  Frames: 33779
pi: [[0.23146151 0.32329017 0.09191758 0.35333073]]
Saving Model
Worker  worker_eval At  61173  Running/Max:  4.695324539719378 11.0  Frames: 38921
pi: [[0.0388166  0.04652858 0.0102864  0.90436834]

Worker  worker_eval At  551027  Running/Max:  30.180877659267388 40.0  Frames: 283850
pi: [[0.11085091 0.45016196 0.394004   0.04498306]]
Saving Model
Worker  worker_eval At  561183  Running/Max:  27.564666449020798 40.0  Frames: 288928
pi: [[0.37897423 0.24491277 0.00795669 0.36815634]]
Saving Model
Worker  worker_eval At  570487  Running/Max:  30.568641841336166 43.0  Frames: 293580
pi: [[0.45829472 0.38440764 0.04879125 0.10850643]]
Saving Model
Worker  worker_eval At  580821  Running/Max:  31.25138591210066 43.0  Frames: 298747
pi: [[0.40882504 0.5782921  0.00076118 0.01212177]]
Saving Model
Worker  worker_eval At  590225  Running/Max:  33.32426032992139 43.0  Frames: 303449
pi: [[0.5408332  0.35281822 0.00443256 0.10191604]]
Saving Model
Worker  worker_eval At  600802  Running/Max:  32.29564720246142 43.0  Frames: 308738
pi: [[8.2223642e-01 1.6955401e-01 7.6194555e-03 5.9016293e-04]]
Saving Model
Worker  worker_eval At  610918  Running/Max:  31.17437412953494 43.0  Frames: 313796


Worker  worker_eval At  1080974  Running/Max:  29.777334785069886 71.0  Frames: 548825
pi: [[3.996998e-04 9.993812e-01 2.182341e-04 8.819557e-07]]
Saving Model
Worker  worker_eval At  1091051  Running/Max:  30.969009352484356 71.0  Frames: 553864
pi: [[1.4031516e-05 9.9998593e-01 8.0739356e-12 1.2821299e-12]]
Saving Model
Worker  worker_eval At  1101147  Running/Max:  31.25136703616499 71.0  Frames: 558913
pi: [[0.5888181  0.16334672 0.08195075 0.16588446]]
Saving Model
Worker  worker_eval At  1110456  Running/Max:  29.453521912427853 71.0  Frames: 563565
pi: [[5.6242421e-02 9.4109094e-01 9.0432685e-04 1.7623843e-03]]
Saving Model
Worker  worker_eval At  1122647  Running/Max:  29.17953015406953 71.0  Frames: 569661
pi: [[2.6270396e-03 9.9501210e-01 8.4178737e-06 2.3523786e-03]]
Saving Model
Worker  worker_eval At  1131181  Running/Max:  30.291877482316686 71.0  Frames: 573928
pi: [[2.4645182e-03 8.9782840e-01 2.0313979e-04 9.9503972e-02]]
Saving Model
Worker  worker_eval At  1140785  R

Worker  worker_eval At  1602460  Running/Max:  39.68176473785537 95.0  Frames: 809572
pi: [[0.15120879 0.05627526 0.19273321 0.5997827 ]]
Saving Model
Worker  worker_eval At  1610239  Running/Max:  37.59500649389658 95.0  Frames: 813462
pi: [[0.00160936 0.9535656  0.00162373 0.04320127]]
Saving Model
Worker  worker_eval At  1622735  Running/Max:  34.42858538458099 95.0  Frames: 819711
pi: [[7.6689024e-04 5.5005943e-04 2.0691172e-04 9.9847609e-01]]
Saving Model
Worker  worker_eval At  1632494  Running/Max:  36.351438745359545 95.0  Frames: 824587
pi: [[3.2584564e-11 1.0000000e+00 2.2647438e-10 4.9486131e-12]]
Saving Model
Worker  worker_eval At  1641020  Running/Max:  36.285198845367105 95.0  Frames: 828852
pi: [[3.7172155e-03 9.9028242e-01 7.0604561e-05 5.9297867e-03]]
Saving Model
Worker  worker_eval At  1652811  Running/Max:  36.715518962445366 95.0  Frames: 834746
pi: [[8.3606768e-01 1.0617544e-01 5.7752300e-02 4.5720476e-06]]
Saving Model
Worker  worker_eval At  1661379  Running/Ma

In [7]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 2, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [8]:
tf.reset_default_graph()
test_method()

INFO:tensorflow:Restoring parameters from saves/breakout/final_R1/checkpoints/model1981886.cptk


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Exception in thread Thread-38:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", li